# W9 - October 18 - Pandas DataFrames

Pandas is a third-party library (not in the Standard Libraray), included with the Anaconda Distribution. Hence, you don't need to install it in the default Anaconda environment (`base`).

**Why use Pandas?**

The `pandas` package provides the DataFrame object, which is basically the equivalent of a spreadsheet table.
- As opposed to `numpy` arrays, DataFrames can contain a mixture of different data types in each column. DataFrames also have named columns and rows (indexes). 
- While dictionaries allow you to work with different data types, `pandas` also contains many functions and methods to manipulate the data inside the DataFrame.

These features are very useful for statistical calculations.

https://pandas.pydata.org/docs/

In [ ]:
import pandas as pd

`pd` is the conventional alias for `pandas`.

## The `pandas` DataFrame

### From arrays

A DataFrame can be created from a dictionary, where each key corresponds to the column name and the values are in the form of `numpy` array or lists.

For example, the projectile motion problem from HW3:

In [ ]:
import numpy as np
from scipy.constants import g

# Create numpy arrays
angles = np.arange(30, 91)
t = 2 * np.sin(np.radians(angles)) / g
h = np.sin(np.radians(angles)) ** 2 / (2 * g)
d = np.sin(2 * np.radians(angles)) / g

df_projectile = pd.DataFrame({"Angles": angles,
                              "Time of Flight": t,
                              "Maximum Height": h,
                              "Range": d})
df_projectile.set_index("Angles", inplace=True)  # inplace keyword is important. Without it, a new DataFrame is created.
df_projectile

### Accessing columns

Columns can be easily accessed, similar to dictionaries

In [ ]:
df_projectile["Range"]

Or as attributes of the DataFrame object

In [ ]:
df_projectile.Range

***Note:*** The latter only works if the column already exists. You cannot set a new column this way. For example:

In [ ]:
# Create a DataFrame
df_example = pd.DataFrame({"A": [0, 1, 2], "B": [7, 13, 42]})
df_example

In [ ]:
# Change the values of column B
df_example["B"] = [10, 50, 99]
df_example

In [ ]:
# Add a column C
df_example["C"] = [100, 255, 924]
df_example

In [ ]:
# Change the values of column B using the attribute
df_example.B = [5555, 2222, 9999]
df_example

In [ ]:
# Add a column D using the attribute
df_example.D = [0, 10, 0]

**Object Types**

In [ ]:
type(df_projectile)

In [ ]:
type(df_projectile.Range)

In [ ]:
df_projectile.columns

In [ ]:
type(df_projectile.columns)

In [ ]:
df_projectile.index

In [ ]:
type(df_projectile.index)

### Access Rows and Columns with `.loc()` and `.iloc()`

Use the **`.loc()`** method to access rows and columns by ***values***.

In [ ]:
df_projectile

`df.loc[row, column]` will return specific values

In [ ]:
df_projectile.loc[90, "Time of Flight"]

In [ ]:
type(df_projectile.loc[90, "Time of Flight"])

You can pass multiple columns as lists

In [ ]:
df_projectile.loc[90, ["Range", "Maximum Height"]]

In [ ]:
type(df_projectile.loc[90, ["Range", "Maximum Height"]])

Passing multiple rows returns a DataFrame

In [ ]:
df_projectile.loc[[45, 90]]

In [ ]:
type(df_projectile.loc[[45, 90]])

In [ ]:
df_projectile.loc[[45, 90], ["Range", "Time of Flight"]]

In [ ]:
type(df_projectile.loc[[45, 90], ["Range", "Time of Flight"]])

Use the **`.iloc()`** method to access rows and columns by ***integer positions***.

In [ ]:
df_projectile

`df.iloc[row]` with a single index returns a Series

In [ ]:
df_projectile.iloc[0]

`df.iloc[row]` with multiple indexes returns a DataFrame

In [ ]:
df_projectile.iloc[[0, 3, 57, 60]]

Can be used to access rows and columns together.

Passing a single column returns a Series.

In [ ]:
df_projectile.iloc[0:11, 2]

Passing multiple columns returns a DataFrame.

In [ ]:
df_projectile.iloc[0:11, 0:2]

### Viewing part of the data

In [ ]:
df_projectile.head()  # Default is 5 rows

In [ ]:
df_projectile.head(7)

In [ ]:
df_projectile.tail(2)

**For a quick summary of the DataFrame**

In [ ]:
df_projectile.describe()

### DataFrames from files

The most common way of creating a DataFrame is by reading a txt, csv or Excel file.

`pandas` also has IO support for JSON, XML, HDF5 and other formats.

For example, the `UTM_data.csv` file from HW3.

In [ ]:
df_UTM = pd.read_csv("UTM_data.csv", index_col="time")  # Indexes do not have to be integers
df_UTM

**Saving to files**

In [ ]:
df_UTM.to_csv("UTM_data_saved.csv")

**Saving as NumPy arrays**

Can easily convert all numerical columns (without the index) to `numpy` arrays, if needed.

In [ ]:
df_UTM.to_numpy()

## Manipulating DataFrames

### Column operations

**Using Boolean masks to clean/filter data**

In [ ]:
threshold = 10
df_UTM_cleaned = df_UTM[df_UTM["force"] > threshold].copy()
df_UTM_cleaned

**Operations on columns**

In [ ]:
displacement_0 = df_UTM_cleaned["displacement"].iloc[0]
displacement_0

In [ ]:
df_UTM_cleaned["displacement"] -= displacement_0
df_UTM_cleaned

In [ ]:
df_UTM_cleaned.index -= df_UTM_cleaned.index[0]
df_UTM_cleaned

**Creating new columns from existing columns**

In [ ]:
area = 4 * 4  # mm^2
l_0 = 6000  # initial length (um)

df_UTM_calc = df_UTM_cleaned.copy()
df_UTM_calc["stress"] = df_UTM_cleaned["force"] / area  # N/mm^2 or MPa
df_UTM_calc["strain"] = df_UTM_cleaned["displacement"] / l_0
df_UTM_calc

In [ ]:
df_UTM_calc.describe()

### Applying functions to DataFrame columns


In [ ]:
df_UTM_func = df_UTM_cleaned.copy()
df_UTM_func["displacement"] -= df_UTM_func["displacement"].iloc[0]
df_UTM_func.index -= df_UTM_func.index[0]
df_UTM_func

**Using `df.apply()`**

In [ ]:
def stress(x):
    area = 4**2  # mm^2
    
    return x / area  # MPa or N/mm^2

df_UTM_func["stress"] = df_UTM_func["force"].apply(stress)
df_UTM_func

Can also be used with `lambda` functions

In [ ]:
df_UTM_func["strain"] = df_UTM_func["Force"].apply(lambda x: x / 6000)
df_UTM_func

**You can also apply functions to the labels**

In [ ]:
df_UTM_func.columns = [col.capitalize() for col in df_UTM_func.columns]
df_UTM_func

## Simple statistics with DataFrames

DataFrames are especially useful when your data also includes categorical values.

In [ ]:
df_soccer = pd.read_excel("Soccer teams.xlsx")
df_soccer

In [ ]:
df_soccer.mean(numeric_only=True)

In [ ]:
df_soccer.max(numeric_only=True)

In [ ]:
# Without numeric_only keyword
df_soccer.max()

In [ ]:
df_soccer.sum(numeric_only=True)

In [ ]:
df_soccer.describe()

### Grouping

In [ ]:
grouped = df_soccer.groupby("League")
for league, df in grouped:
    print(league)
    print(df)
type(df)

In [ ]:
type(grouped)

In [ ]:
grouped.size()

In [ ]:
grouped.mean()

In [ ]:
grouped.max()

In [ ]:
grouped.sum()

### Sorting

`by` list of columns, in order of priority.

In [ ]:
# By default, pandas returns a new DataFrame. The original is left untouched. This is a safety feature.
df_soccer.sort_values(by=["Founded", "Trophies"])
df_soccer

In [ ]:
df_soccer.sort_values(by=["Founded", "Trophies"], inplace=True)  # inplace argument changes the current DataFrame.
df_soccer

In [ ]:
df_soccer.sort_values(by="Trophies", ascending=False, inplace=True)
df_soccer

In [ ]:
df_reset_without_dropping = df_soccer.reset_index(inplace=True)  # The old indexes are added as a new column by default
df_reset_without_dropping

In [ ]:
df_soccer.reset_index(drop=True, inplace=True)
df_soccer

### Indexing min/max values

In [ ]:
founded_min = df_soccer["Founded"].idxmin()
print(f"The oldest club in this dataset is {df_soccer.loc[founded_min, 'Team Name']}, founded in {df_soccer.loc[founded_min, 'Founded']}.")

In [ ]:
founded_max = df_soccer["Founded"].idxmax()
print(f"The youngest club in this dataset is {df_soccer.loc[founded_max, 'Team Name']}, founded in {df_soccer.loc[founded_max, 'Founded']}.")

**With the grouped data:**

In [ ]:
grouped = df_soccer.groupby("League")  # Need to group again as the index was reset above.
for league, df in grouped:
    founded_min = df["Founded"].idxmin()
    founded_max = df["Founded"].idxmax()
    print(f"In {league}, the oldest club is {df.loc[founded_min, 'Team Name']}, founded in {df.loc[founded_min, 'Founded']}."
          f"\nThe youngest club is {df.loc[founded_max, 'Team Name']}, founded in {df.loc[founded_max, 'Founded']}.\n")